In [1]:
import os, glob, math, operator
import ROOT as ROOT
from ROOT import TCanvas, TColor, TGaxis, TH1F, TPad, TFile, TGraphAsymmErrors,TLatex,TLine,gStyle,TLegend,gROOT,TGraph
from ROOT import kBlack, kBlue, kRed
from array import array
import matplotlib.pyplot as plt, numpy as np
from matplotlib import text
from matplotlib.colors import LogNorm
import datetime
import pandas as pd 

def SetCanvas():
    c = ROOT.TCanvas("", "", 650, 600)
    c.SetBottomMargin(0.050)
    c.SetRightMargin(0.050)
    c.SetLeftMargin(0.050)
    c.SetTopMargin(0.050)
    return c

def SetCMSAxis(h, xoffset=1., yoffset=1.):
    h.GetXaxis().SetTitleSize(0.047)
    h.GetYaxis().SetTitleSize(0.047)
    if type(h) is ( (not ROOT.TGraphAsymmErrors) or (not ROOT.TGraph)):
        h.GetZaxis().SetTitleSize(0.047)

    h.GetXaxis().SetLabelSize(0.047)
    h.GetYaxis().SetLabelSize(0.047)
    if type(h) is ( (not ROOT.TGraphAsymmErrors) or (not ROOT.TGraph)):
        h.GetZaxis().SetLabelSize(0.047)

    h.GetXaxis().SetTitleOffset(xoffset)
    h.GetYaxis().SetTitleOffset(yoffset)
    
    h.GetYaxis().CenterTitle()
    return h

def SetLegend(coordinate_=[.50,.65,.90,.90],ncol=2):
    c_=coordinate_
    legend=ROOT.TLegend(c_[0], c_[1],c_[2],c_[3])
    legend.SetBorderSize(0)
    legend.SetNColumns(ncol)
    legend.SetLineColor(1)
    legend.SetLineStyle(1)
    legend.SetLineWidth(1)
    legend.SetFillColor(0)
    legend.SetFillStyle(0)
    legend.SetTextFont(42)
    legend.SetTextSize(0.055)
    return legend


def drawenergy1D(is2017, text_="Work in progress 2018", data=True):
    #pt = ROOT.TPaveText(0.0877181,0.9,0.9580537,0.96,"brNDC")
    pt = ROOT.TPaveText(0.0997181,0.95,0.9580537,0.96,"brNDC")
    pt.SetBorderSize(0)
    pt.SetTextAlign(12)
    pt.SetFillStyle(0)
    pt.SetTextFont(52)

    cmstextSize = 0.07
    preliminarytextfize = cmstextSize * 0.7
    lumitextsize = cmstextSize *0.7
    pt.SetTextSize(cmstextSize)
    text = pt.AddText(0.03,0.57,"#font[60]{CMS}")

    #pt1 = ROOT.TPaveText(0.0877181,0.9,0.9580537,0.96,"brNDC")
    pt1 = ROOT.TPaveText(0.0877181,0.95,0.9580537,0.96,"brNDC")
    pt1.SetBorderSize(0)
    pt1.SetTextAlign(12)
    pt1.SetFillStyle(0)
    pt1.SetTextFont(52)

    pt1.SetTextSize(preliminarytextfize)
    #text1 = pt1.AddText(0.215,0.4,text_)
    text1 = pt1.AddText(0.15,0.4,text_)

    #pt2 = ROOT.TPaveText(0.0877181,0.9,0.9580537,0.96,"brNDC")
    pt2 = ROOT.TPaveText(0.0997181,0.95,0.9580537,0.96,"brNDC")
    pt2.SetBorderSize(0)
    pt2.SetTextAlign(12)
    pt2.SetFillStyle(0)
    pt2.SetTextFont(52)
    pt2.SetTextFont(42)
    pt2.SetTextSize(lumitextsize)

    pavetext = ''
    if is2017 and data: pavetext = str(luminosity_)+' fb^{-1}'+" (13 TeV)"
    if (not is2017) and data: pavetext = str(luminosity_)+' fb^{-1}'+"(13 TeV)"

    if is2017 and not data: pavetext = "13 TeV"
    if (not is2017) and not data: pavetext = "13 TeV"

    if data: text3 = pt2.AddText(0.68,0.5,pavetext)
    if not data: text3 = pt2.AddText(0.85,0.5,pavetext)

    return [pt,pt1,pt2]

def getLatex():
    latex =  TLatex()
    latex.SetNDC();
    latex.SetTextSize(0.04);
    latex.SetTextAlign(31);
    latex.SetTextAlign(11);
    latex.SetTextColor(1);
    return latex


def ExtraText(text_, x_, y_):
    if not text_:
        print("nothing provided as text to ExtraText, function crashing")
    ltx = ROOT.TLatex(x_, y_, text_)

    if len(text_) > 0:
        ltx.SetTextFont(42)
        ltx.SetTextSize(0.049)
        #ltx.Draw(x_,y_,text_)
        ltx.Draw('same')
    return ltx


def getGraph(n,x,y,lc,mc,ms):
    gr =TGraph(n,x,y)
    gr.SetFillColor(4)
    #gr.SetFillStyle(3004)
    gr.SetLineColor(4)
    gr.SetLineWidth(2)
    gr.SetMarkerStyle(ms)
    gr.SetMarkerSize(1.5)
    gr.SetLineColor(lc)
    gr.SetLineWidth(1)
    gr.SetMarkerColor(mc)
    gr.GetYaxis().SetTitle("Signal Efficiency")
    gr.GetXaxis().SetTitle("M_{a} (GeV)")
    return gr
    
def getHisto(hist,ls,lc,mc,ms):
    gr = hist#.Clone('gr')
    gr.SetLineStyle(ls)
    gr.SetLineWidth(2)
    gr.SetMarkerStyle(ms)
    gr.SetMarkerSize(1)
    gr.SetLineColor(lc)
    gr.SetMarkerColor(mc)
    # gr.Scale(1/gr.Integral())
    return gr

ROOT.gROOT.SetBatch(True)
ROOT.gStyle.SetFrameLineWidth(3)
ROOT.gStyle.SetOptTitle(0)
ROOT.gStyle.SetOptStat(0)
ROOT.gStyle.SetLegendBorderSize(0)
ROOT.gROOT.ForceStyle(1)
ROOT.gStyle.SetImageScaling(3.)

Welcome to JupyROOT 6.24/06


In [2]:
luminosity_1 = '{0:.2f}'.format(41.5)
luminosity_2 = '{0:.2f}'.format(35.81)
lumi = {}
lumi['2017'] = luminosity_1
lumi['2016'] = luminosity_2

inDir = {}
# inDir['NLO'] = '/Users/ptiwari/cernBox/Documents/ExoPieCapper/plots_norm/v17_12-00-04_pfMetCorrPt_NLO_v2/bbDMRoot'
# inDir['LO'] = '/Users/ptiwari/cernBox/Documents/ExoPieCapper/plots_norm/v17_12-00-04_pfMetCorrPt_TnLonly/bbDMRoot'
inDir['NLO'] = '/Users/ptiwari/cmsCERN/Documents/ExoPieCapper/plots_norm/v18_12-01-00_oldSF/bbDMRoot'
inDir['LO'] = '/Users/ptiwari/cmsCERN/Documents/ExoPieCapper/plots_norm/v18_12-01-00_115ele_newSF/bbDMRoot'
regions = ['ZeeCR_2j', 'ZeeCR_3j', 'WenuCR_1b', 'TopenuCR_2b']
histos = 'lep1_pT'

for reg in regions:
    file = {}
    file['NLO'] = ROOT.TFile(inDir['NLO']+'/h_reg_'+reg+'_'+histos+'.root')
    file['LO'] = ROOT.TFile(inDir['LO']+'/h_reg_'+reg+'_'+histos+'.root')
    
    plot_bkgSum = {}
    plot_data = {}
    # plot_bkgSum['NLO'] = file['NLO'].Get('DYJets')
    # plot_bkgSum['LO'] = file['LO'].Get('DYJets')
    plot_bkgSum['NLO'] = file['NLO'].Get('data_obs')
    plot_bkgSum['LO'] = file['LO'].Get('data_obs')
    # plot_data['NLO'] = file['NLO'].Get('data_obs')
    # plot_data['LO'] = file['LO'].Get('data_obs')
    
    c1 = SetCanvas()
    ##Upper PAD##
    c1_1 = ROOT.TPad("c1_1", "c1_1", 0., 0.25, 1., 1.)
    c1_1.SetBottomMargin(0.001)
    c1_1.SetTopMargin(0.10)
    c1_1.SetLeftMargin(0.18)
    c1_1.SetRightMargin(0.05)
    c1_1.SetLogy(1)
    c1_1.SetGrid()
    c1_1.Draw()
    c1_1.cd()
    legend = SetLegend([.65,.65,.85,.85],ncol=1)
    ls,lc,mc,ms =1,1,1,1
    histos_dict = {}
    for key in plot_bkgSum:
        histo = getHisto(plot_bkgSum[key],ls,lc,mc,ms+20)
        histo = SetCMSAxis(histo)
        legend.AddEntry(histo, str(key)+' DYJets', "PEL")
        lc+=1;mc+=1;ms+=1
        histo.SetMaximum(histo.GetMaximum()*50)
        histo.GetYaxis().SetTitle("Events")
        histo.GetYaxis().SetTitleSize(0.08)
        histo.GetYaxis().SetTitleOffset(1.2)
        histo.GetYaxis().SetMoreLogLabels()
        histo.GetXaxis().SetTitle("MET (GeV)")
        histo.Draw("Hist E0 same")
        histos_dict.update({key: histo})

    # for key in plot_data:
    #     histo = getHisto(plot_data[key],ls,lc,mc,ms+20)
    #     legend.AddEntry(histo,str(key)+' data',"PEL")
    #     lc+=1;mc+=1;ms+=1
    #     histo.Draw("Hist E0 same")
    legend.Draw('L same')
    c1_1.Update()  
    c1.cd()
    ##Lower PAD##
    c1_2 = ROOT.TPad("c1_2", "c1_2", 0., 0., 1., 0.25)
    c1_2.SetLeftMargin(0.18)
    c1_2.SetRightMargin(0.05)
    c1_2.SetTopMargin(0.00)
    c1_2.SetBottomMargin(0.38)
    c1_2.SetGrid()
    c1_2.Draw()
    c1_2.cd()
    h_clone = histos_dict['NLO'].Clone()
    # h_clone.Add(p1_2,-1)
    h_clone.Divide(histos_dict['LO'])
    # h_clone = getHisto(h_clone,ls,lc,20+mc,ms)
    h_clone.GetYaxis().SetTitle("NLO/LO")
    h_clone.GetXaxis().SetTitle(histos)
    h_clone.GetXaxis().SetTitleSize(0.18)
    h_clone.GetYaxis().SetTitleSize(0.15)
    h_clone.GetYaxis().SetTitleOffset(0.5)
    h_clone.GetXaxis().SetLabelSize(0.15)
    h_clone.GetYaxis().SetLabelSize(0.15)
    h_clone.SetLineStyle(1)
    h_clone.SetMaximum(1.25)
    h_clone.SetMinimum(0.75)
    h_clone.GetYaxis().SetNdivisions(505)
    h_clone.Draw("hist e0")
    c1_2.Update()
    c1.cd()
    t2d = ExtraText(str(reg).replace('_','(')+')', 0.30, 0.85)
    t2d.SetTextSize(0.06)
    t2d.SetTextAlign(12)
    t2d.SetNDC(ROOT.kTRUE)
    t2d.SetTextFont(42)
    t2d.Draw("same")
    pt = drawenergy1D(True,text_="    Internal",data=False)
    for ipt in pt: ipt.Draw()
    latex=getLatex()
    c1.Update()
    c1.Draw()
    c1.SaveAs(histos+'_'+reg+'_2017.pdf')
    c1.Close()


KeyError: 'NLO'